In [125]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
# Read in the data
df = pd.read_csv('GlobalWeatherRepository.csv')

In [126]:
df [['location_name', 'temperature_celsius', 'wind_kph', 'humidity', 'pressure_mb', 'precip_mm', 'cloud']].head(10)
#print 10 rows of interesting information

,location_name,temperature_celsius,wind_kph,humidity,pressure_mb,precip_mm,cloud
0,Kabul,28.8,11.5,19,1004.0,0.0,0
1,Tirana,27.0,6.1,54,1006.0,0.0,75
2,Algiers,28.0,13.0,30,1014.0,0.0,25
3,Andorra La Vella,10.2,9.7,51,1015.0,0.0,6
4,Luanda,25.0,3.6,69,1016.0,0.0,75
5,Saint John's,29.0,15.1,79,1015.0,0.3,75
6,Buenos Aires,9.0,11.2,71,1023.0,0.0,0
7,Yerevan,31.0,9.0,26,1003.0,0.0,25
8,Canberra,13.0,15.1,62,1017.0,0.0,0
9,Vienna,16.0,19.1,82,1005.0,0.0,75


In [127]:
median_values = df[['temperature_celsius', 'wind_kph', 'humidity', 'pressure_mb', 'precip_mm', 'cloud']]
print(median_values)

       temperature_celsius  wind_kph  humidity  pressure_mb  precip_mm  cloud
0                     28.8      11.5        19       1004.0       0.00      0
1                     27.0       6.1        54       1006.0       0.00     75
2                     28.0      13.0        30       1014.0       0.00     25
3                     10.2       9.7        51       1015.0       0.00      6
4                     25.0       3.6        69       1016.0       0.00     75
...                    ...       ...       ...          ...        ...    ...
16754                 34.0       6.8        56       1014.0       0.32     50
16755                 20.9       3.6        82       1019.0       0.00     58
16756                 16.0       9.7        33       1019.0       0.00      0
16757                 24.3       7.6        54       1011.0       0.00      6
16758                 20.8      19.8        62       1013.0       0.19     87

[16759 rows x 6 columns]


In [128]:
from adjust_dataset import adjust_dataset

df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')

df = adjust_dataset(df, 'Stockholm')

# Selecting the features and the target variable
features = ['wind_kph','humidity','pressure_mb']
X = df[features]
y = df['temperature_celsius']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.03, random_state=0)

c:\Users\OMEN\Documents\Iot_JM\adjust_dataset.py:43: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['moonset'].replace('No moonset', method='ffill', inplace=True)
c:\Users\OMEN\Documents\Iot_JM\adjust_dataset.py:44: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['moonrise'].replace('No moonrise', method='ffill', inplace=True)
c:\Users\OMEN\Documents\Iot_JM\adjust_dataset.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df['sunrise'])
c:\Users\OMEN\Documents\Iot_JM\adjust_dataset.py:48: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunset'] =

In [129]:
# Number of lag days If it is possible would be interesting to test to make the last days more valuable as they probably have a closer connection to the upcoming day.
lag_days = 10

# Create lagged features for temperature
for i in range(1, lag_days + 1):
    df[f'temperature_celsius_lag_{i}'] = df['temperature_celsius'].shift(i)


In [130]:
df = df.dropna()


In [131]:
features = [f'temperature_celsius_lag_{i}' for i in range(1, lag_days + 1)] + ['wind_kph', 'humidity', 'pressure_mb']
X = df[features]
y = df['temperature_celsius']


In [132]:
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [133]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [134]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 10.979621311146945
